<a href="https://colab.research.google.com/github/JackEllie/ComfyUI_Colab/blob/main/%E3%80%8CComfyUI_Colab%E3%80%8Dby_jackellie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI Colab TW




### 使用流程

1. 到**前置步驟**, **下載模型**, **Extensions**, **其他設置**進行設置
2. 左上角工具列 > 執行階段 > 變更執行階段類型 > 硬體加速器改為 GPU
3. 左上角工具列 > 執行階段 > 全部執行
4. 到**啟動 WebUI** 點選 public URL


### 參考資料
[Webui版本](
https://colab.research.google.com/drive/1aEARTRAzNWBvoT3Bv_pI506zn1HTcH1E?usp=sharing#scrollTo=A6OFCYOzNjDX)

### 更新時間
2024/08/14

#本ComfyUI執行腳本由[JackEllie](https://www.youtube.com/@JackEllie)維護
任何使用疑問請到Discord上方提問

Discord邀請碼: https://discord.gg/TM5d89YNwA

## 1 - 前置步驟

In [ ]:
# @title 1.1 安裝 Comfy-cil&ComfyUI
%cd /content

# @markdown ---
# @markdown - [Comfy-cil](https://github.com/Comfy-Org/comfy-cli)
! apt -y install -qq aria2
! pip install -q jedi comfy-cli pyngrok -U
!npm install -g localtunnel > /dev/null 2>&1

# 安裝ComfyUI同時建置環境

! comfy --skip-prompt --workspace=./ComfyUI install --nvidia > /dev/null 2>&1

#%cd /content/SDWCT

In [ ]:
# @title 1.2 建立雲端資料夾
%cd /content
# @markdown 雲端資料夾名稱
data_dir_name = "Stable_Diffusion_WebUI_Colab_TW"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"
# @markdown 不連接 models/Stable-diffusion 資料夾模式
sd_no_link_mode = False # @param {type:"boolean"}

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
comfy_config_path = f"{data_dir}/default"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(comfy_config_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)
os.makedirs(f"{models_path}/ControlNet", exist_ok=True)
os.makedirs(f"{models_path}/custom_nodes", exist_ok=True)
os.makedirs

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf ComfyUI/models/embeddings && ln -s {models_path}/embeddings ComfyUI/models/embeddings
    elif dir != "ControlNet":
        if dir == "Stable-diffusion" and sd_no_link_mode == True:
            ! rm -Rf ComfyUI/models/checkpoints/Stable-diffusion && mkdir ComfyUI/models/checkpoints/Stable-diffusion
            continue
        elif dir == "Stable-diffusion" and sd_no_link_mode == False:
            ! rm -Rf ComfyUI/models/checkpoints/Stable-diffusion && ln -s {models_path}/{dir} ComfyUI/models/checkpoints/Stable-diffusion
            continue
        ! rm -Rf ComfyUI/models/{dir} && ln -s {models_path}/{dir} ComfyUI/models/{dir}

! rm -Rf ComfyUI/outputs && ln -s {data_dir}/outputs ComfyUI/outputs
! mkdir -p ComfyUI/user && rm -Rf ComfyUI/user/default && ln -s {data_dir}/default ComfyUI/user/default


! touch {custom_nodes_path}



/content
Mounted at /content/drive


## 2 - 下載模型

In [ ]:
# @title 2.1 下載SD模型
# @markdown 需要下載SD模型組合包嗎？
need_to_download_sd = True # @param {type:"boolean"}
# @markdown 選擇SD模型組合包
model_package = "clandestine-xl-1.0" # @param ["clandestine-xl-1.0","Dark Sushi 2.5D", "AnythingV4.5", "AbyssOrangeMix3", "CounterfeitV2.5", "PastalMix", "CamelliaMix_2.5D", "ChilloutMix_Ni_fix", "其他(填寫sd_url和vae_url)"]
# @markdown 其他SD模型下載網址
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - 除了利用Colab下載模型，推薦使用雲端硬碟捷徑以節省空間。
# @markdown  - [[AI繪圖] WebUI Colab玩家小技巧：使用雲端硬碟捷徑節省空間](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - Anything, AbyssOrangeMix, Counterfeit 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > anime.vae.pt
# @markdown - CamelliaMix_2.5D 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > vae-ft-mse-840000-ema-pruned.safetensors
# @markdown - PastalMix 和 ChilloutMix 不需要設定 VAE
# @markdown  - SD VAE > None

sd_urls = []
vae_urls = []
if model_package == "clandestine-xl-1.0":
    sd_urls.append("https://huggingface.co/yodayo-ai/clandestine-xl-1.0/resolve/main/clandestine-xl-1.0.safetensors?download=true")
elif model_package == "AnythingV4.5":
    sd_urls.append("https://huggingface.co/xyn-ai/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI/resolve/main/VAE/anime.vae.pt")
elif model_package == "Dark Sushi 2.5D":
    sd_urls.append("https://civitai.com/api/download/models/141866")
elif model_package == "AbyssOrangeMix3":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI/resolve/main/VAE/anime.vae.pt")
elif model_package == "CounterfeitV2.5":
    sd_urls.append("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI/resolve/main/VAE/anime.vae.pt")
elif model_package == "PastalMix":
    sd_urls.append("https://huggingface.co/JamesFlare/pastel-mix/resolve/main/pastelmix-better-vae-fp32.safetensors")
elif model_package == "CamelliaMix_2.5D":
    sd_urls.append("https://huggingface.co/WuLing/SD-WebUI/resolve/main/SD/CamelliaMix_2.5D/camelliamix25D_v10.safetensors")
    vae_urls.append("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")
elif model_package == "ChilloutMix_Ni_fix":
    sd_urls.append("https://civitai.com/api/download/models/11745")
elif model_package == "其他(填寫sd_url和vae_url)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

if sd_urls != [] and need_to_download_sd == True:
    %cd /content/ComfyUI/models/checkpoints/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != [] and need_to_download_sd == True:
    %cd /content/ComfyUI/models/VAE
    for vae_url in vae_urls:
        if vae_url != "":
            ! wget -nc --content-disposition {vae_url}

In [ ]:
# @title 2.2 下載LoRA Embedding模型

# @markdown Embedding
EasyNegative = False # @param {type:"boolean"}
embedding_urls = ""  # @param {type:"string"}


# @markdown LoRA/LyCORIS
lora_urls = ""  # @param {type:"string"}


# @markdown ---
# @markdown - [EasyNegative](https://civitai.com/models/7808/easynegative)
# @markdown   - 泛用負面 embedding
# @markdown   - 觸發詞為 EasyNegative, 加在負面 prompt
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
# @markdown - [LoRA預覽圖](https://i.imgur.com/XooSV8p.png)請自行添加，格式為：`模型名稱.png`

embeddings = []
embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if EasyNegative == True:
    embeddings.append("https://civitai.com/api/download/models/9208")
if embedding_urls != ['']:
    embeddings = embeddings + embedding_urls

if embeddings != []:
    %cd /content/SDWCT/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}


loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd /content/SDWCT/models/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}



## 3 - custom_nodes

---



In [ ]:
# @title 3.1 安裝自訂節點
# @markdown #常見節點
ComfyUI_Impact_Pack = True # @param {type:"boolean"}
rgthree_comfy = True # @param {type:"boolean"}
ComfyUI_KJNodes = True # @param {type:"boolean"}
comfyui_controlnet_aux = True # @param {type:"boolean"}
ComfyUI_Advanced_ControlNet = True # @param {type:"boolean"}
ComfyUI_Custom_Scripts = True # @param {type:"boolean"}
ComfyUI_TiledDiffusion = True # @param {type:"boolean"}
cg_use_everywhere = True # @param {type:"boolean"}
ComfyUI_UltimateSDUpscale = True # @param {type:"boolean"}
ComfyUI_Custom_Nodes_AlekPet = False # @param {type:"boolean"}
ComfyUI_IPAdapter_plus = False # @param {type:"boolean"}
ComfyUI_Easy_Use = False # @param {type:"boolean"}
Derfuu_ComfyUI_ModdedNodes = False # @param {type:"boolean"}
was_node_suite_comfyui = False # @param {type:"boolean"}
comfy_mtb = False # @param {type:"boolean"}
ComfyUI_Comfyroll_CustomNodes = False # @param {type:"boolean"}
AIGODLIKE_COMFYUI_TRANSLATION = False # @param {type:"boolean"}
ComfyUI_essentials = False # @param {type:"boolean"}


# @markdown ---
# @markdown # 選用節點
ComfyUI_InstantID = False # @param {type:"boolean"}
efficiency_nodes_comfyui = False # @param {type:"boolean"}
comfyui_tooling_nodes = False # @param {type:"boolean"}
ComfyUI_WD14_Tagger = False # @param {type:"boolean"}
ComfyUI_SUPIR = False # @param {type:"boolean"}
ComfyUI_VideoHelperSuite = False # @param {type:"boolean"}
canvas_tab = False # @param {type:"boolean"}
ComfyUI_APISR_KJ = False # @param {type:"boolean"}
ComfyUI_DynamiCrafterWrapper = False # @param {type:"boolean"}
ComfyUI_Frame_Interpolation = False # @param {type:"boolean"}
ComfyUI_IC_Light_Native = False # @param {type:"boolean"}
ComfyUI_layerdiffusion = False # @param {type:"boolean"}
ComfyUI_AI_Assistant = False # @param {type:"boolean"}
ComfyUI_Logic = False # @param {type:"boolean"}

custon_name = ""  # @param {type:"string"}

# @markdown ---
# @markdown ## 由於自訂節點眾多請選擇需要的節點就好否則`安裝會非常緩慢`在這邊沒安裝到的節點可再使用中透過管理器再次添加
# @markdown 若要新增節點請在`custon_name`選項中輸入節點名稱(以github上的名稱為主)

# @markdown EX:[ComfyUI-Impact-Pack](https://github.com/ltdrdata/ComfyUI-Impact-Pack)請輸入ComfyUI-Impact-Pack(要看那個節點庫的專案名稱)並以逗號分隔

custom_nodes_pair = [
            (ComfyUI_Impact_Pack, f"ComfyUI-Impact-Pack"),
            (rgthree_comfy, f"rgthree-comfy"),
            (ComfyUI_KJNodes, f"ComfyUI-KJNodes"),
            (comfyui_controlnet_aux, f"comfyui_controlnet_aux"),
            (ComfyUI_Advanced_ControlNet, f"ComfyUI-Advanced-ControlNet"),
            (ComfyUI_Custom_Nodes_AlekPet, f"ComfyUI_Custom_Nodes_AlekPet"),
            (ComfyUI_IPAdapter_plus, f"ComfyUI_IPAdapter_plus"),
            (ComfyUI_Custom_Scripts, f"ComfyUI-Custom-Scripts"),
            (ComfyUI_Easy_Use, f"ComfyUI-Easy-Use"),
            (ComfyUI_TiledDiffusion, f"ComfyUI-TiledDiffusion"),
            (Derfuu_ComfyUI_ModdedNodes, f"Derfuu_ComfyUI_ModdedNodes"),
            (was_node_suite_comfyui, f"was-node-suite-comfyui"),
            (cg_use_everywhere, f"cg-use-everywhere"),
            (comfy_mtb, f"comfy_mtb"),
            (ComfyUI_Comfyroll_CustomNodes, f"ComfyUI_Comfyroll_CustomNodes"),
            (AIGODLIKE_COMFYUI_TRANSLATION, f"AIGODLIKE-COMFYUI-TRANSLATION"),
            (ComfyUI_essentials, f"ComfyUI_essentials"),
            (ComfyUI_UltimateSDUpscale, f"ComfyUI_UltimateSDUpscale"),
            (ComfyUI_Logic, f"ComfyUI-Logic"),
            (ComfyUI_InstantID, f"ComfyUI_InstantID"),
            (efficiency_nodes_comfyui, f"efficiency-nodes-comfyui"),
            (comfyui_tooling_nodes, f"comfyui-tooling-nodes"),
            (ComfyUI_WD14_Tagger, f"ComfyUI-WD14-Tagger"),
            (ComfyUI_SUPIR, f"ComfyUI-SUPIR"),
            (ComfyUI_VideoHelperSuite, f"ComfyUI-VideoHelperSuite"),
            (canvas_tab, f"canvas_tab"),
            (ComfyUI_APISR_KJ, f"ComfyUI-APISR-KJ"),
            (ComfyUI_DynamiCrafterWrapper, f"ComfyUI-DynamiCrafterWrapper"),
            (ComfyUI_Frame_Interpolation, f"ComfyUI-Frame-Interpolation"),
            (ComfyUI_IC_Light_Native, f"ComfyUI-IC-Light-Native"),
            (ComfyUI_AI_Assistant, f"ComfyUI_AI_Assistant"),
            (ComfyUI_layerdiffusion, f"ComfyUI-layerdiffusion")


          ]


custom_nodes = []

for name, rename in custom_nodes_pair:
    if name == True and rename not in custom_nodes:
        custom_nodes.append(rename)

if ComfyUI_IPAdapter_plus == True :
  !pip install -q insightface onnx onnxruntime onnxruntime-gpu

import os

if custom_nodes != []:
    for custom_node in custom_nodes:
        custom_nodesn_name, _ = os.path.splitext(custom_node.split("/")[-1])
        if not os.path.exists(custom_nodesn_name):
            print("install node "+custom_node)
            ! comfy node install {custom_node} > /dev/null 2>&1


install node ComfyUI-Impact-Pack
install node rgthree-comfy
install node ComfyUI-KJNodes
install node comfyui_controlnet_aux
install node ComfyUI-Advanced-ControlNet
install node ComfyUI-Custom-Scripts
install node ComfyUI-TiledDiffusion
install node cg-use-everywhere
install node ComfyUI_UltimateSDUpscale


In [ ]:
# @title 3.2 下載 SD1.5ControlNet 模型
# @markdown 不連接 models/ControlNet 資料夾模式
ControlNet_no_link_mode = True # @param {type:"boolean"}
# @markdown ---
# @markdown ## SD1.5 ControlNet
SD15_ip2p = False # @param {type:"boolean"}
SD15_shuffle = False # @param {type:"boolean"}
SD15_depth = False # @param {type:"boolean"}
SD15_canny = False # @param {type:"boolean"}
SD15_inpaint = False # @param {type:"boolean"}
SD15_lineart = False # @param {type:"boolean"}
SD15_mlsd = False # @param {type:"boolean"}
SD15_normalbae = False # @param {type:"boolean"}
SD15_openpose = False # @param {type:"boolean"}
SD15_scribble = False # @param {type:"boolean"}
SD15_seg = False # @param {type:"boolean"}
SD15_softedge = False # @param {type:"boolean"}
SD15_lineart_anime = False # @param {type:"boolean"}
SD15_tile = False # @param {type:"boolean"}

# @markdown ---
# @markdown ## IP Adapter
ip_adapter_plus_sd15 = False # @param {type:"boolean"}
ip_adapter_sd15_light = False # @param {type:"boolean"}

# @markdown ---
# @markdown - 除了利用Colab下載模型，推薦使用雲端硬碟捷徑以節省空間。
# @markdown  - [[AI繪圖] WebUI Colab玩家小技巧：使用雲端硬碟捷徑節省空間](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - 多重 ControlNet 請到 Settings > ControlNet > Multi ControlNet 設置
# @markdown - 參考資料
# @markdown  - https://github.com/lllyasviel/ControlNet-v1-1-nightly
# @markdown  - https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace
# @markdown  - https://github.com/TencentARC/T2I-Adapter
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)

import os


if ControlNet_no_link_mode == True:

    %cd /content/ComfyUI/models/controlnet
else:
    %cd {models_path}/ControlNet
if SD15_ip2p == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors

if SD15_shuffle == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors

if SD15_depth == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors

if SD15_canny == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors

if SD15_inpaint == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors

if SD15_lineart == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors

if SD15_mlsd == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors

if SD15_normalbae == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors

if SD15_openpose == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors

if SD15_scribble == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors

if SD15_seg == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors

if SD15_softedge == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors

if SD15_lineart_anime == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors

if SD15_tile == True:
    ! wget -nc --content-disposition https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors

if ip_adapter_plus_sd15 == True:
    ! wget -nc --content-disposition https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors

if ip_adapter_plus_sd15 == True:
    ! wget -nc --content-disposition https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors


%cd /content/ComfyUI/models

for model in os.listdir(f"{models_path}/ControlNet"):
    ! rm -f controlnet/{model} && ln -s {models_path}/ControlNet/{model} controlnet/{model}

/content/ComfyUI/models/controlnet
/content/ComfyUI/models


In [ ]:
# @title 3.3 下載 SDXL ControlNet 模型
# @markdown 不連接 models/ControlNet 資料夾模式
ControlNet_no_link_mode = True # @param {type:"boolean"}
# @markdown ---
# @markdown ## SDXL ControlNet
# @markdown ### 如果需要使用SDXL的ControlNet建議使用SDXL union即可union支援多種輸入包含openpose、Lineart等可以參考這支[影片](https://youtu.be/Zq5YjLX0KU0)
AnytestV3 = False # @param {type:"boolean"}
AnytestV4 = False # @param {type:"boolean"}
SDXL_union = False # @param {type:"boolean"}
SDXL_inpaint = False # @param {type:"boolean"}
SDXL_lineart = False # @param {type:"boolean"}
SDXL_softedge = False # @param {type:"boolean"}

# @markdown ---
# @markdown ## IP Adapter
InstantID = False # @param {type:"boolean"}
ip_adapter = False # @param {type:"boolean"}
ip_adapter_faceid = False # @param {type:"boolean"}



import os
sdxlcnurl= "https://huggingface.co/Jackellie/All_ControlNet_models/resolve/main/"


if ControlNet_no_link_mode == True:
    %cd /content/ComfyUI/models/controlnet
else:
    %cd {models_path}/ControlNet
if AnytestV3 == True:
    ! wget -nc --content-disposition {sdxlcnurl}anytest/CN-anytest_v3-50000_fp16.safetensors

if AnytestV4 == True:
    ! wget -nc --content-disposition {sdxlcnurl}anytest/CN-anytest_v4-marged.safetensors

if SDXL_union == True:
    ! wget -nc --content-disposition {sdxlcnurl}union/controlnet%2B%2B_union_promax_sdxl.safetensors

if SDXL_inpaint == True:
    ! wget -nc --content-disposition {sdxlcnurl}inpaint/Kataragi_inpaintXL-fp16.safetensors

if SDXL_lineart == True:
    ! wget -nc --content-disposition {sdxlcnurl}lineart/Katarag_lineartXL-fp16.safetensors

if SDXL_softedge == True:
    ! wget -nc --content-disposition {sdxlcnurl}softedge/mistoLine_softedge_sdxl_fp16.safetensors

if InstantID == True:
    ! wget -nc --content-disposition {sdxlcnurl}InstantID/control_instant_id_sdxl.safetensors
    ! wget -nc --content-disposition {sdxlcnurl}InstantID/ip-adapter_instant_id_sdxl.bin

if ip_adapter == True:
    ! wget -nc --content-disposition {sdxlcnurl}ipadapter/ip-adapter-plus_sdxl_vit-h.safetensors

if ip_adapter_faceid == True:
    ! wget -nc --content-disposition {sdxlcnurl}ipadapter/ip-adapter-faceid-plusv2_sdxl.bin
    %cd /content/ComfyUI/models/Lora
    ! wget -nc --content-disposition https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors
    if ControlNet_no_link_mode == True:
      %cd /content/ComfyUI/models/controlnet
    else:
      %cd {models_path}/ControlNet


%cd /content/ComfyUI/models

for model in os.listdir(f"{models_path}/ControlNet"):
    ! rm -f controlnet/{model} && ln -s {models_path}/ControlNet/{model} controlnet/{model}

In [ ]:
# @title 3.3 下載檔案到指定資料夾
target_dir = "" # @param {type:"string"}
file_urls = "" # @param {type:"string"}
# @markdown ---
# @markdown - 將指定資料夾填在 `target_dir`，檔案的下載網址填在 `file_urls`。
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
file_urls = list(map(str.strip, file_urls.split(",")))

import os
if os.path.exists(os.path.split(target_dir)[0]):
    os.makedirs(target_dir, exist_ok=True)
    %cd {target_dir}
    for url in file_urls:
        if url != '':
            ! wget -nc --content-disposition {url}

In [ ]:
# @title 3.4 自訂資料夾連結
google_drive_dir = "" # @param {type:"string"}
comfyui_dir = "" # @param {type:"string"}
# @markdown ---
# @markdown - 將雲端硬碟資料夾填在 `google_drive_dir`，WebUI資料夾填在 `comfyui_dir`。

import os
if os.path.exists(os.path.split(comfyui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(comfyui_dir):
        ! cp -Rf {comfyui_dir}/* {google_drive_dir}
    else:
        os.makedirs(comfyui_dir, exist_ok=True)
    ! rm -Rf {comfyui_dir} && ln -s {google_drive_dir} {comfyui_dir}

## 4 - 其他設置

In [ ]:
# @title 4.1 命令行參數
# @markdown 命令行參數
front_end_version = True # @param {type:"boolean"}
# @markdown SSH Tunnel

ssh_tunnel = "localtunnel" # @param ["ngrok(推薦，需填寫ngrok_authtoken)", "localtunnel"]
ngrok_authtoken = "" # @param {type:"string"}

if ssh_tunnel== "ngrok(推薦，需填寫ngrok_authtoken)":
  if ngrok_authtoken != "":
    !ngrok config add-authtoken {ngrok_authtoken}
  else:
    print("Please enter ngrok_authtoken")
# @markdown ---
# @markdown ngrok_authtoken 填寫方法
# @markdown - 註冊 [ngrok](https://ngrok.com/) 帳號
# @markdown - Your Authtoken > Copy
# @markdown - ssh_tunnel 選 ngrok > 貼上 ngrok_authtoken
# @markdown - 等 local URL 出來之後再點 ngrok URL


if front_end_version == True:
    arg = "--front-end-version Comfy-Org/ComfyUI_frontend@latest"


## 5 - 啟動 ComfyUI

In [ ]:
# @title 5.1 啟動 ComfyUI
# @markdown - 點選 URL 開啟 ComfyUI<br>

# @markdown - ### ngrok
# @markdown ![img](https://i.imgur.com/BQOhqga.png)
# @markdown - ### localtunnel請複製IP位址後點選下方網址貼上IP位址後啟動
# @markdown ![img](https://i.imgur.com/38lIu4t.png)



import subprocess
import threading
import time
import socket
import urllib.request
from pyngrok import ngrok, conf

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\nComfyUI finished loading, trying to launch Ngrok\n")
    if ssh_tunnel=="ngrok(推薦，需填寫ngrok_authtoken)":
      try:
          public_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

      except Exception as e:
          print("Failed to get public IP:", e)
          return

      # 使用 pyngrok 來啟動 ngrok 服務
      http_tunnel = ngrok.connect(port, bind_tls=True)
      print(f"Ngrok Tunnel URL: {http_tunnel.public_url}")
    else:
      print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
      p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
      for line in p.stdout:
        print(line.decode(), end='')
# 啟動執行緒來等待服務並啟動 ngrok
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()


!comfy launch -- --dont-print-server {arg}